In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
%load_ext google.colab.data_table

Authenticated


Stock Price Project

In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (select s.*, row_number() over(order by date) as index
from `jrjames83-1171.sampledata.stock_prices` s
order by 1)

select bt.*, avg(bt.Close) over(order by bt.index range between 49 preceding and current row) fifty_ma,
avg(bt.Close) over(order by bt.index range between 199 preceding and current row) twohundred_ma
from base_table bt
limit 5

,Date,Close,index,fifty_ma,twohundred_ma
0,2014-01-02 16:00:00+00:00,556.55,1,556.550000,556.550000
1,2014-01-03 16:00:00+00:00,552.50,2,554.525000,554.525000
2,2014-01-06 16:00:00+00:00,558.66,3,555.903333,555.903333
3,2014-01-07 16:00:00+00:00,569.42,4,559.282500,559.282500
4,2014-01-08 16:00:00+00:00,570.61,5,561.548000,561.548000


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (select s.*, row_number() over(order by date) as index
from `jrjames83-1171.sampledata.stock_prices` s
order by 1),

averages as (
    select bt.*, avg(bt.Close) over(order by bt.index range between 49 preceding and current row) fifty_ma,
    avg(bt.Close) over(order by bt.index range between 199 preceding and current row) twohundred_ma
    from base_table bt
)

select a.*, a.fifty_ma > a.twohundred_ma signal -- true / false
from averages a
limit 5

,Date,Close,index,fifty_ma,twohundred_ma,signal
0,2014-01-02 16:00:00+00:00,556.55,1,556.550000,556.550000,False
1,2014-01-03 16:00:00+00:00,552.50,2,554.525000,554.525000,False
2,2014-01-06 16:00:00+00:00,558.66,3,555.903333,555.903333,False
3,2014-01-07 16:00:00+00:00,569.42,4,559.282500,559.282500,False
4,2014-01-08 16:00:00+00:00,570.61,5,561.548000,561.548000,False


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (select s.*, row_number() over(order by date) as index
from `jrjames83-1171.sampledata.stock_prices` s
order by 1),

averages as (
    select bt.*, avg(bt.Close) over(order by bt.index range between 49 preceding and current row) fifty_ma,
    avg(bt.Close) over(order by bt.index range between 199 preceding and current row) twohundred_ma
    from base_table bt
)

select a.*, if (a.fifty_ma > a.twohundred_ma, 'Buy', 'Sell') signal 
from averages a
limit 5

,Date,Close,index,fifty_ma,twohundred_ma,signal
0,2014-01-02 16:00:00+00:00,556.55,1,556.550000,556.550000,Sell
1,2014-01-03 16:00:00+00:00,552.50,2,554.525000,554.525000,Sell
2,2014-01-06 16:00:00+00:00,558.66,3,555.903333,555.903333,Sell
3,2014-01-07 16:00:00+00:00,569.42,4,559.282500,559.282500,Sell
4,2014-01-08 16:00:00+00:00,570.61,5,561.548000,561.548000,Sell


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (select s.*, row_number() over(order by date) as index
from `jrjames83-1171.sampledata.stock_prices` s
order by 1),

averages as (
    select bt.*, avg(bt.Close) over(order by bt.index range between 49 preceding and current row) fifty_ma,
    avg(bt.Close) over(order by bt.index range between 199 preceding and current row) twohundred_ma
    from base_table bt
),

signals as (select a.*, if (a.fifty_ma > a.twohundred_ma, 'Buy', 'Sell') signal 
from averages a)

select s.*, lag(s.signal) over (order by s.date) prev_signal
from signals s
order by 1 
limit 5

,Date,Close,index,fifty_ma,twohundred_ma,signal,prev_signal
0,2014-01-02 16:00:00+00:00,556.55,1,556.550000,556.550000,Sell,None
1,2014-01-03 16:00:00+00:00,552.50,2,554.525000,554.525000,Sell,Sell
2,2014-01-06 16:00:00+00:00,558.66,3,555.903333,555.903333,Sell,Sell
3,2014-01-07 16:00:00+00:00,569.42,4,559.282500,559.282500,Sell,Sell
4,2014-01-08 16:00:00+00:00,570.61,5,561.548000,561.548000,Sell,Sell


In [ ]:
%%bigquery --project august-bucksaw-330517
with base_table as (select s.*, row_number() over(order by date) as index
from `jrjames83-1171.sampledata.stock_prices` s
order by 1),

averages as (
    select bt.*, avg(bt.Close) over(order by bt.index range between 49 preceding and current row) fifty_ma,
    avg(bt.Close) over(order by bt.index range between 199 preceding and current row) twohundred_ma
    from base_table bt
),

signals as (select a.*, if (a.fifty_ma > a.twohundred_ma, 'Buy', 'Sell') signal 
from averages a)

select * from (
    select s.*, lag(s.signal) over (order by s.date) <> s.signal changed_signal
    from signals s
    order by 1)
where changed_signal is true
limit 5

,Date,Close,index,fifty_ma,twohundred_ma,signal,changed_signal
0,2014-03-17 16:00:00+00:00,596.04,51,586.0686,585.489804,Buy,True
1,2014-04-29 16:00:00+00:00,526.26,81,573.0126,573.269383,Sell,True
2,2014-07-30 16:00:00+00:00,585.81,145,566.8014,566.086483,Buy,True
3,2014-10-27 16:00:00+00:00,539.29,207,566.1070,566.295850,Sell,True
4,2015-04-30 16:00:00+00:00,537.34,334,548.0516,547.971150,Buy,True
